**Классификация изображений с помощью сверточных нейронных сетей**

Получилось с помощью нескольких сверток добится accuracy 30%, но углубление нейронной сети или увеличение количества сверток не принесли улучшений результата.

In [522]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.python.client import device_lib

In [523]:
#conda install -c conda-forge cudatoolkit cudnn
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14332458214539056473
 xla_global_id: -1]

In [524]:
tf.config.list_physical_devices('GPU')

[]

In [525]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [526]:
# Выполните загрузку модели
(X_train, y_train), (X_val, y_val) = tf.keras.datasets.cifar100.load_data()

In [527]:
# Преобразуйте метки классов в one_hot формат
y_train = tf.keras.utils.to_categorical(y_train,num_classes=CLASSES)
y_val = tf.keras.utils.to_categorical(y_val,num_classes=CLASSES)
y_train[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [528]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

In [529]:
# Задайте архитектуру модели
model = tf.keras.models.Sequential([
    
    tf.keras.Input(shape=[32,32,3]),
    tf.keras.layers.Conv2D(20, kernel_size=(2, 2),strides=(2, 2), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(20, kernel_size=(2, 2),strides=(2, 2), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),

    #tf.keras.layers.Conv2D(20, kernel_size=(2, 2),strides=(2, 2), activation='relu'),
    #tf.keras.layers.BatchNormalization(),

    #tf.keras.layers.Dropout(0.05),

    #tf.keras.layers.Conv2D(15, kernel_size=(3, 3),strides=(1, 1), activation='relu'),
    #tf.keras.layers.BatchNormalization(),
    #tf.keras.layers.Conv2D(15, kernel_size=(3, 3),strides=(1, 1), activation='relu'),
    #tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Flatten(),
    #tf.keras.layers.Dense(120, activation='relu'),
    #tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(CLASSES, activation='softmax'),



    
])

In [530]:
# вычисление количества операций
from keras_flops import get_flops
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")

FLOPs: 0.6037e6


In [531]:
# вывод краткой информации о модели
model.summary()

Model: "sequential_44"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_110 (Conv2D)         (None, 16, 16, 20)        260       
                                                                 
 batch_normalization_109 (Ba  (None, 16, 16, 20)       80        
 tchNormalization)                                               
                                                                 
 conv2d_111 (Conv2D)         (None, 8, 8, 20)          1620      
                                                                 
 batch_normalization_110 (Ba  (None, 8, 8, 20)         80        
 tchNormalization)                                               
                                                                 
 dropout_44 (Dropout)        (None, 8, 8, 20)          0         
                                                                 
 flatten_44 (Flatten)        (None, 1280)            

In [532]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=1000, decay_rate=0.5)
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [533]:
# обучения модели
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
    ],
    use_multiprocessing=True,
    workers=16,
    epochs=256
)

Epoch 1/256
391/391 [==============================] - 10s 21ms/step - loss: 3.7617 - accuracy: 0.1727 - val_loss: 3.2668 - val_accuracy: 0.2322
Epoch 2/256
391/391 [==============================] - 8s 21ms/step - loss: 3.0405 - accuracy: 0.2697 - val_loss: 3.0673 - val_accuracy: 0.2705
Epoch 3/256
391/391 [==============================] - 8s 20ms/step - loss: 2.7986 - accuracy: 0.3164 - val_loss: 2.9345 - val_accuracy: 0.2912
Epoch 4/256
391/391 [==============================] - 8s 20ms/step - loss: 2.6309 - accuracy: 0.3487 - val_loss: 2.9534 - val_accuracy: 0.2920
Epoch 5/256
391/391 [==============================] - 8s 19ms/step - loss: 2.5072 - accuracy: 0.3731 - val_loss: 2.8901 - val_accuracy: 0.3068
Epoch 6/256
391/391 [==============================] - 8s 19ms/step - loss: 2.4012 - accuracy: 0.3963 - val_loss: 2.8529 - val_accuracy: 0.3114
Epoch 7/256
391/391 [==============================] - 8s 20ms/step - loss: 2.3221 - accuracy: 0.4117 - val_loss: 2.9198 - val_accuracy

KeyboardInterrupt: 